In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import plotly.figure_factory as ff

In [ ]:
import os
import pandas as pd
from pathlib import Path

# Путь к папке с данными
data_dir = Path('data')

# Получаем все Excel файлы
excel_files = list(data_dir.glob('*.xlsx')) + list(data_dir.glob('*.xls'))

print(f"Найдено {len(excel_files)} Excel файлов\n")

# Нужные колонки для СУ
su_columns_needed = [
    'Дата замера',
    'Ток фазы A',
    'Коэффициент загрузки',
    'Выходная частота',
    'Давление на приеме насоса',
    'Температура двигателя',
    'Устьевое давление'
]

# Словари для хранения всех данных
all_agzu = []
all_su = []

# Читаем каждый файл
for i, file_path in enumerate(excel_files, 1):
    # Извлекаем номер скважины (имя файла без расширения)
    well_number = file_path.stem
    print(f"[{i}/{len(excel_files)}] Обрабатываю скважину: {well_number}")
    
    # Сначала проверяем какие листы есть в файле
    try:
        xls = pd.ExcelFile(file_path)
        available_sheets = xls.sheet_names
    except Exception as e:
        print(f"  ✗ Не удалось открыть файл: {e}")
        continue
    
    # Читаем АГЗУ (все колонки)
    if 'Замеры АГЗУ из ТМ' in available_sheets:
        try:
            df_agzu = pd.read_excel(file_path, sheet_name='Замеры АГЗУ из ТМ')
            df_agzu['well_number'] = well_number
            all_agzu.append(df_agzu)
            print(f"  ✓ АГЗУ: {len(df_agzu)} строк, {len(df_agzu.columns)} колонок")
        except Exception as e:
            print(f"  ✗ АГЗУ ошибка: {e}")
    else:
        print(f"  ⚠️ АГЗУ: лист не найден")
    
    # Читаем СУ (только нужные колонки)
    if 'Замеры СУ' in available_sheets:
        try:
            df_su_full = pd.read_excel(file_path, sheet_name='Замеры СУ')
            
            # Оставляем только нужные колонки
            cols_to_keep = [col for col in su_columns_needed if col in df_su_full.columns]
            
            df_su = df_su_full[cols_to_keep].copy()
            df_su['well_number'] = well_number
            
            all_su.append(df_su)
            print(f"  ✓ СУ: {len(df_su)} строк, оставлено {len(cols_to_keep)} колонок из {len(df_su_full.columns)}")
            
            # Показываем какие колонки не найдены
            missing = [col for col in su_columns_needed if col not in df_su_full.columns]
            if missing:
                print(f"     ⚠️ Не найдены: {missing}")
                
        except Exception as e:
            print(f"  ✗ СУ ошибка: {e}")
    else:
        print(f"  ⚠️ СУ: лист не найден")
    
    print()  # Пустая строка для читаемости

# Объединяем все данные в один DataFrame
print("="*60)
if all_agzu:
    df_agzu_combined = pd.concat(all_agzu, ignore_index=True)
    print(f"✅ Всего данных АГЗУ: {len(df_agzu_combined)} строк из {len(all_agzu)} файлов")
    print(f"   Колонки: {list(df_agzu_combined.columns)}")
else:
    df_agzu_combined = pd.DataFrame()
    print("⚠️ Данных АГЗУ не найдено")

if all_su:
    df_su_combined = pd.concat(all_su, ignore_index=True)
    print(f"✅ Всего данных СУ: {len(df_su_combined)} строк из {len(all_su)} файлов")
    print(f"   Колонки: {list(df_su_combined.columns)}")
else:
    df_su_combined = pd.DataFrame()
    print("⚠️ Данных СУ не найдено")

print(f"\n📊 Уникальные скважины: {sorted(df_agzu_combined['well_number'].unique())}")

In [ ]:
# Заполняем пропущенные значения в df_su_combined для каждой скважины отдельно
print("До заполнения пропущенных значений:")
print("Количество пропущенных значений в каждой колонке:")
print(df_su_combined.isnull().sum())
print("\n")

# Создаем копию для заполнения
df_su_filled = df_su_combined.copy()

# Получаем список всех колонок кроме well_number и даты
date_col = 'Дата замера'
cols_to_fill = [col for col in df_su_filled.columns if col not in [date_col, 'well_number']]

print(f"Колонки для заполнения: {cols_to_fill}\n")

# Заполняем пропуски для каждой скважины отдельно
for well in df_su_filled['well_number'].unique():
    print(f"Заполняю пропуски для скважины {well}...")
    
    # Получаем индексы строк этой скважины
    mask = df_su_filled['well_number'] == well
    
    # Заполняем каждую колонку
    for col in cols_to_fill:
        if col not in df_su_filled.columns:
            continue
            
        before_null = df_su_filled.loc[mask, col].isnull().sum()
        
        # Проверяем есть ли хоть одно не-NaN значение
        if df_su_filled.loc[mask, col].notna().sum() == 0:
            print(f"  ⚠️ {col}: все значения NaN, пропускаю")
            continue
        
        # Сначала forward fill - используем ffill() вместо fillna(method='ffill')
        df_su_filled.loc[mask, col] = df_su_filled.loc[mask, col].ffill()
        
        # Потом backward fill - используем bfill() вместо fillna(method='bfill')
        df_su_filled.loc[mask, col] = df_su_filled.loc[mask, col].bfill()
        
        after_null = df_su_filled.loc[mask, col].isnull().sum()
        
        if before_null > 0 and after_null == 0:
            print(f"  ✓ {col}: заполнено {before_null} NaN")
        elif after_null < before_null:
            print(f"  ✓ {col}: было {before_null} NaN, осталось {after_null}")

print("\nПосле заполнения пропущенных значений:")
print("Количество пропущенных значений в каждой колонке:")
print(df_su_filled.isnull().sum())
print("\n")

# Показываем статистику по скважинам
print("Статистика по скважинам:")
for well in sorted(df_su_filled['well_number'].unique()):
    well_data = df_su_filled[df_su_filled['well_number'] == well]
    null_count = well_data.isnull().sum().sum()
    if null_count > 0:
        print(f"  Скважина {well}: {len(well_data)} строк, ⚠️ {null_count} пропущенных значений")
        # Показываем какие именно колонки с NaN
        null_cols = well_data.isnull().sum()
        null_cols = null_cols[null_cols > 0]
        for col_name, count in null_cols.items():
            print(f"     - {col_name}: {count} NaN")
    else:
        print(f"  Скважина {well}: {len(well_data)} строк, ✅ все данные заполнены")

In [ ]:
anomalies = pd.read_excel('./альма/Общая таблица.xlsx', sheet_name = 'Ненормальная работа')
anomalies

In [ ]:
# Парсим периоды аномалий из таблицы
import re

def parse_datetime_range(datetime_str):
    """Извлекает начало и конец из строки типа '11.12.24 01:22 - 18.12.24 21:35'"""
    try:
        # Ищем паттерн даты и времени
        pattern = r'(\d{2}\.\d{2}\.\d{2})\s+(\d{2}:\d{2})\s*-\s*(\d{2}\.\d{2}\.\d{2})\s+(\d{2}:\d{2})'
        match = re.search(pattern, str(datetime_str))
        
        if match:
            start_date, start_time, end_date, end_time = match.groups()
            start = f"20{start_date.split('.')[2]}-{start_date.split('.')[1]}-{start_date.split('.')[0]} {start_time}:00"
            end = f"20{end_date.split('.')[2]}-{end_date.split('.')[1]}-{end_date.split('.')[0]} {end_time}:00"
            return start, end
        return None, None
    except:
        return None, None

# Проверяем парсинг на первой строке
test_start, test_end = parse_datetime_range(anomalies.iloc[0]['Дата и время'])
print(f"Тест парсинга: {test_start} -> {test_end}")

In [ ]:
# Анализируем каждую аномалию
results = []

# Преобразуем даты в datetime
date_col_agzu = df_agzu_combined.columns[0]
date_col_su = df_su_filled.columns[0]

df_agzu_combined[date_col_agzu] = pd.to_datetime(df_agzu_combined[date_col_agzu])
df_su_filled[date_col_su] = pd.to_datetime(df_su_filled[date_col_su])

print(f"Обрабатываю {len(anomalies)} аномалий...\n")

for idx, row in anomalies.iterrows():
    well_number = str(row['Скважина'])
    anomaly_name = row['Название предупреждения']
    datetime_range = row['Дата и время']
    
    print(f"[{idx+1}/{len(anomalies)}] Скважина {well_number}: {anomaly_name}")
    
    # Парсим даты
    start_str, end_str = parse_datetime_range(datetime_range)
    
    if not start_str or not end_str:
        print(f"  ⚠️ Не удалось распарсить даты: {datetime_range}")
        continue
    
    start_dt = pd.to_datetime(start_str)
    end_dt = pd.to_datetime(end_str)
    
    print(f"  📅 Период: {start_dt} -> {end_dt}")
    
    # Фильтруем данные по скважине
    agzu_well = df_agzu_combined[df_agzu_combined['well_number'] == well_number].copy()
    su_well = df_su_filled[df_su_filled['well_number'] == well_number].copy()
    
    if len(agzu_well) == 0 and len(su_well) == 0:
        print(f"  ⚠️ Нет данных для скважины {well_number}")
        continue
    
    # Данные за период аномалии
    mask_agzu_period = (agzu_well[date_col_agzu] >= start_dt) & (agzu_well[date_col_agzu] <= end_dt)
    period_agzu = agzu_well[mask_agzu_period]
    
    mask_su_period = (su_well[date_col_su] >= start_dt) & (su_well[date_col_su] <= end_dt)
    period_su = su_well[mask_su_period]
    
    # Данные за 3 дня ДО аномалии
    before_start = start_dt - pd.Timedelta(days=3)
    mask_agzu_before = (agzu_well[date_col_agzu] >= before_start) & (agzu_well[date_col_agzu] < start_dt)
    before_agzu = agzu_well[mask_agzu_before]
    
    mask_su_before = (su_well[date_col_su] >= before_start) & (su_well[date_col_su] < start_dt)
    before_su = su_well[mask_su_before]
    
    print(f"  📊 АГЗУ: до={len(before_agzu)}, период={len(period_agzu)}")
    print(f"  📊 СУ: до={len(before_su)}, период={len(period_su)}")
    
    # Результат для этой аномалии
    result_row = {
        'Номер скважины': well_number,
        'Причина остановки': anomaly_name,
        'Время аномалии': datetime_range,
        'Начало': start_dt,
        'Конец': end_dt,
    }
    
    # Анализируем АГЗУ параметры
    if len(period_agzu) > 0 and len(before_agzu) > 0:
        for col in agzu_well.columns:
            if col in [date_col_agzu, 'well_number']:
                continue
            
            try:
                before_mean = pd.to_numeric(before_agzu[col], errors='coerce').mean()
                period_mean = pd.to_numeric(period_agzu[col], errors='coerce').mean()
                
                if pd.notna(before_mean) and pd.notna(period_mean):
                    if before_mean != 0:
                        # Обычный расчет процента
                        change_percent = ((period_mean - before_mean) / before_mean) * 100
                        result_row[f'АГЗУ_{col}_изм%'] = round(change_percent, 2)
                    elif period_mean == 0:
                        # Если было 0 и осталось 0 → нет изменений
                        result_row[f'АГЗУ_{col}_изм%'] = 0.0
                    else:
                        # Если было 0, а стало не-0 → 100% (условный рост)
                        result_row[f'АГЗУ_{col}_изм%'] = 100.0
                else:
                    result_row[f'АГЗУ_{col}_изм%'] = None
            except:
                result_row[f'АГЗУ_{col}_изм%'] = None
    
    # Анализируем СУ параметры
    if len(period_su) > 0 and len(before_su) > 0:
        for col in su_well.columns:
            if col in [date_col_su, 'well_number']:
                continue
            
            try:
                before_mean = pd.to_numeric(before_su[col], errors='coerce').mean()
                period_mean = pd.to_numeric(period_su[col], errors='coerce').mean()
                
                if pd.notna(before_mean) and pd.notna(period_mean):
                    if before_mean != 0:
                        # Обычный расчет процента
                        change_percent = ((period_mean - before_mean) / before_mean) * 100
                        result_row[f'СУ_{col}_изм%'] = round(change_percent, 2)
                    elif period_mean == 0:
                        # Если было 0 и осталось 0 → нет изменений
                        result_row[f'СУ_{col}_изм%'] = 0.0
                    else:
                        # Если было 0, а стало не-0 → 100% (условный рост)
                        result_row[f'СУ_{col}_изм%'] = 100.0
                else:
                    result_row[f'СУ_{col}_изм%'] = None
            except:
                result_row[f'СУ_{col}_изм%'] = None
    
    results.append(result_row)
    print(f"  ✅ Обработано\n")

# Создаем итоговую таблицу
df_anomaly_analysis = pd.DataFrame(results)
print("="*80)
print(f"✅ Создана таблица с {len(df_anomaly_analysis)} аномалиями")
df_anomaly_analysis

In [ ]:
# Сохраняем результат в Excel
output_path = './альма/Анализ_аномалий_результат.xlsx'
df_anomaly_analysis.to_excel(output_path, index=False)
print(f"✅ Результат сохранён в {output_path}")

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Получаем все колонки с изменениями в процентах
change_columns = [col for col in df_anomaly_analysis.columns if '_изм%' in col]

# Готовим данные для тепловой карты
heatmap_data = df_anomaly_analysis[['Номер скважины', 'Время аномалии'] + change_columns].copy()

# Создаем метки для строк (скважина + время)
heatmap_data['Метка'] = heatmap_data['Номер скважины'].astype(str) + ' | ' + \
                        heatmap_data['Время аномалии'].astype(str)

# Устанавливаем метку как индекс
heatmap_data = heatmap_data.set_index('Метка')[change_columns]

# Упрощаем названия колонок
simplified_cols = []
for col in heatmap_data.columns:
    # Убираем префиксы и суффиксы, делаем короче
    new_name = col.replace('АГЗУ_', 'А: ').replace('СУ_', 'С: ').replace('_изм%', '')
    # Еще сокращаем длинные названия
    new_name = new_name.replace('Планируемая продолжительность, мин', 'План.продолж.')
    new_name = new_name.replace('Фактическая продолжительность???, мин', 'Факт.продолж.')
    new_name = new_name.replace('Объемный дебит жидкости, м3/сут', 'Дебит жидк.')
    new_name = new_name.replace('Объемный дебит газа, м3/сут', 'Дебит газа')
    new_name = new_name.replace('Давление в коллекторе ГЗУ при замере', 'Давл.ГЗУ')
    new_name = new_name.replace('Давление на приеме насоса', 'Давл.приема')
    new_name = new_name.replace('Температура двигателя', 'Темп.двиг.')
    new_name = new_name.replace('Устьевое давление', 'Устьев.давл.')
    new_name = new_name.replace('Выходная частота', 'Частота')
    simplified_cols.append(new_name)

heatmap_data.columns = simplified_cols

# Ограничиваем значения от -25 до +25 для отображения
# Но сохраняем реальные значения для hover
z_values_display = np.clip(heatmap_data.values, -25, 25)
z_values_real = heatmap_data.values

# Создаем тепловую карту
fig = go.Figure(data=go.Heatmap(
    z=z_values_display,  # Для цвета используем ограниченные значения
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='RdYlGn_r',  # Красный=рост, Зеленый=падение
    zmid=0,  # Центр на нуле
    zmin=-25,  # Минимум шкалы
    zmax=25,   # Максимум шкалы
    text=z_values_real,  # Для текста показываем реальные значения
    texttemplate='%{text:.1f}%',
    textfont={"size": 8},
    colorbar=dict(
        title="Изменение, %",
        tickvals=[-25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25],
        ticktext=['-25%', '-20%', '-15%', '-10%', '-5%', '0%', '5%', '10%', '15%', '20%', '25%']
    ),
    hoverongaps=False,
    hovertemplate='<b>%{y}</b><br>%{x}<br><b>Изменение: %{text:.2f}%</b><extra></extra>',
    customdata=z_values_real
))

fig.update_layout(
    title='Изменения параметров при аномалиях<br><sub>Красный - рост параметра, Зеленый - падение параметра (шкала: -25% до +25%)</sub>',
    xaxis_title='Параметры',
    yaxis_title='Скважина | Время аномалии',
    height=max(700, len(heatmap_data) * 30),
    width=1600,
    xaxis={'tickangle': -45, 'side': 'bottom'},
    yaxis={'tickfont': {'size': 9}},
    font={'size': 10}
)

fig.show()